In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


In [ ]:
from dotenv import load_dotenv
import os
import getpass


# Load environment variables from .env file
load_dotenv()

# Access groq_key
groq_key = os.getenv("GROQ_API_KEY")

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass(groq_key)

In [16]:
!pip install -qU langchain-groq

In [17]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

### Basic ChatPromptTemplate

In [ ]:
# Define a prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{question}")
])

# Chain it together
chain = prompt | llm | StrOutputParser()

# Use it
response = chain.invoke({"question": "What's the capital of Egypt?"})
print(response)

The capital of Egypt is Cairo.


### Prompt with Multiple Variables

In [9]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant that summarizes documents."),
    ("human", "Please summarize this text in {style} style:\n\n{text}")
])

chain = prompt | llm | StrOutputParser()

response = chain.invoke({
    "text": "The Nile is the longest river in the world. It flows through multiple countries in Africa.",
    "style": "bullet point"
})

print(response)

Here is a summary of the text in bullet point style:

• The Nile is the longest river in the world.
• It flows through multiple countries in Africa.


### Few-Shot Prompting (with examples)

In [10]:
from langchain.prompts import FewShotChatMessagePromptTemplate

examples = [
    {
        "input": "2 + 2",
        "output": "4"
    },
    {
        "input": "10 * 3",
        "output": "30"
    }
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a calculator."),
    few_shot_prompt,
    ("human", "{input}")
])

chain = final_prompt | llm | StrOutputParser()
response = chain.invoke({"input": "7 * 6"})
print(response)


42


In [13]:
from langchain_core.output_parsers import StrOutputParser

# Without bind.
chain = (
    llm
    | StrOutputParser()
)

chain.invoke("Repeat quoted words exactly: 'One two three four five.'")
# Output is 'One two three four five.'



"'One two three four five.'"

In [14]:
# With bind.
chain = (
    llm.bind(stop=["three"])
    | StrOutputParser()
)

chain.invoke("Repeat quoted words exactly: 'One two three four five.'")
# Output is 'One two'

"'One two "